In [1]:
%load_ext autoreload

In [2]:
%autoreload 3

In [1]:
from src.eda import load_data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score #, mean_squared_error

In [2]:
from src.features import *

In [3]:
rides  = pd.read_csv('./data/rides_info.csv')
cars = pd.read_csv('./data/car_train.csv')
drivers = pd.read_csv('./data/driver_info.csv')

In [4]:
path = './data/car_test.csv'
test = pd.read_csv(path)

In [6]:
def get_all_features(rides, drivers):
    rides = add_simple_features(rides)
    rides_drivers = rides.merge(drivers, on='user_id', how='left')
    rides_drivers_feats = get_rides_n_drivers_features(rides_drivers)
    rides_drivers_feats = get_rides_features(rides_drivers_feats, rides)
    return rides_drivers_feats


In [7]:
features = get_all_features(rides, drivers)
train = cars.merge(features, on='car_id', how='left')
test = test.merge(features, on='car_id', how='left')

(391001, 10)
(4250, 33)
(4250, 41)


### Try catboost

In [9]:
features2drop = ['car_id']
targets = ['target_class', 'target_reg']
cat_features = ['car_type', 'fuel_type', 'model']

filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

print('cat_features :', len(cat_features), cat_features)
print('num_features :', len(num_features), num_features)
print('targets', targets)

cat_features : 3 ['car_type', 'fuel_type', 'model']
num_features : 55 ['car_rating', 'year_to_start', 'riders', 'year_to_work', 'user_rating_avg_avg', 'user_time_accident_avg_avg', 'rating_avg_avg', 'speed_max_avg_avg', 'speed_avg_avg_avg', 'user_ride_quality_avg_avg', 'deviation_normal_avg_avg', 'log_distance_delta_avg_avg', 'user_rating_avg_std', 'user_time_accident_avg_std', 'rating_avg_std', 'speed_max_avg_std', 'speed_avg_avg_std', 'user_ride_quality_avg_std', 'deviation_normal_avg_std', 'log_distance_delta_avg_std', 'user_rating_avg_min', 'user_time_accident_avg_min', 'rating_avg_min', 'speed_max_avg_min', 'speed_avg_avg_min', 'user_ride_quality_avg_min', 'deviation_normal_avg_min', 'log_distance_delta_avg_min', 'user_rating_avg_max', 'user_time_accident_avg_max', 'rating_avg_max', 'speed_max_avg_max', 'speed_avg_avg_max', 'user_ride_quality_avg_max', 'deviation_normal_avg_max', 'log_distance_delta_avg_max', 'deviation_normal_avg_last_1d', 'user_ride_quality_avg_last_1d', 'deviat

In [10]:
X = train[filtered_features].drop(targets, axis=1, errors='ignore')
y = train['target_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = CatBoostClassifier(cat_features=cat_features)


clf.fit(
    X_train, y_train, 
    eval_set=(X_test, y_test),
    verbose=150, plot=False
)

y_pred = clf.predict(X_test)
print(f'\n Accuracy: {accuracy_score(y_test, y_pred)}')

Learning rate set to 0.109335
0:	learn: 1.5781396	test: 1.5887869	best: 1.5887869 (0)	total: 166ms	remaining: 2m 46s
150:	learn: 0.0166330	test: 0.0227686	best: 0.0227686 (150)	total: 9.79s	remaining: 55.1s
300:	learn: 0.0077993	test: 0.0145329	best: 0.0145329 (300)	total: 19.3s	remaining: 44.9s
450:	learn: 0.0050619	test: 0.0112459	best: 0.0112459 (450)	total: 28.8s	remaining: 35.1s
600:	learn: 0.0033766	test: 0.0088321	best: 0.0088321 (600)	total: 38.4s	remaining: 25.5s
750:	learn: 0.0026938	test: 0.0078416	best: 0.0078416 (750)	total: 48.3s	remaining: 16s
900:	learn: 0.0021651	test: 0.0069962	best: 0.0069962 (900)	total: 58s	remaining: 6.37s
999:	learn: 0.0019163	test: 0.0065083	best: 0.0065083 (999)	total: 1m 4s	remaining: 0us

bestTest = 0.006508256952
bestIteration = 999


 Accuracy: 1.0


In [11]:
fi = clf.get_feature_importance(prettified=True)
fi

,Feature Id,Importances
0,speed_max_p90,23.053608
1,user_time_accident_avg_max,13.619374
2,speed_avg_avg_avg,12.802134
3,user_rating_avg_avg,9.172437
4,user_time_accident_avg_avg,8.576191
5,log_distance_delta_avg_avg,5.346764
6,user_time_accident_avg_std,5.295748
7,speed_avg_p85,4.900603
8,speed_max_avg_std,3.235386
9,speed_max_avg_avg,3.002889


In [12]:
test['target_class'] = clf.predict(test[filtered_features]).ravel()
test[['car_id', 'target_class']].to_csv('./subs/drivers_n_rides_feats_v3.csv', index=False)
print('CSV file saved!')

CSV file saved!
